In [4]:
import re
import pandas as pd
import numpy as np

from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, SmallInteger, BigInteger, Float, String, Date, Time, DateTime 

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)


def connect_db(dbname, host, port, user, password):
    conn_string = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
    db = create_engine(conn_string)
    return db.connect()

def change_col_names(s: str) -> str:
    s = s.strip()
    s = ''.join([' '+s[i] if s[i].isupper() and (s[i-1].islower() if i>0 else False) else s[i] for i in range(len(s))]).lstrip(' ')
    s = s.capitalize()
    s = re.sub('[^a-zA-Z0-9]', '_', s)
    return ''.join(['_'+i.lower() if i.isupper() else i for i in s]).lstrip('_')

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

HOST = 'camara-dos-deputados.cjdcxj8gvuxr.us-east-2.rds.amazonaws.com'
DB = 'camara_deputados'
LOGIN = os.environ['POSTGRES_USER']
PASSWORD = os.environ['POSTGRES_PASS']

db_connection = connect_db(
        dbname=DB,
        host=HOST,
        port="5432",
        user=LOGIN,
        password=PASSWORD
)        

In [11]:
# Deputados
df_deputados = pd.read_csv(Path("data/extrator_files/deputados.csv"), sep=';')
df_deputados.columns = [change_col_names(i) for i in df_deputados.columns]
df_deputados.insert(0, 'id_deputado', df_deputados['uri'].str.split('/').str[-1])
df_deputados.drop(columns=['uri', 'cpf', 'url_rede_social', 'url_website'], inplace=True)

# Tratando tipos de dados
df_deputados = df_deputados[df_deputados['id_legislatura_final'] == 56] # Legislatura 56 é a atual. TODO: Deixar dinâmico
df_deputados['id_deputado'] = df_deputados['id_deputado'].astype(int)
df_deputados['nome'] = df_deputados['nome'].str.title()
df_deputados['nome_civil'] = df_deputados['nome_civil'].str.title()
df_deputados['data_nascimento'] = pd.to_datetime(df_deputados['data_nascimento'], format='%Y-%m-%d')
df_deputados['data_falecimento'] = pd.to_datetime(df_deputados['data_falecimento'], format='%Y-%m-%d')
df_deputados['url_foto'] = df_deputados.agg(lambda x: f"https://www.camara.leg.br/internet/deputado/bandep/{x['id_deputado']}.jpg", axis=1)

# Profissoes do deputado
df_deputados_profissoes = pd.read_csv(Path("data/extrator_files/deputadosProfissoes.csv"), sep=';')
df_deputados_profissoes.columns = [change_col_names(i) for i in df_deputados_profissoes.columns]
df_deputados_profissoes = df_deputados_profissoes[['id', 'titulo']]
df_deputados_profissoes.rename(columns={"id": "id_deputado", "titulo": "profissao"}, inplace=True)
df_deputados_profissoes = df_deputados_profissoes.groupby("id_deputado").aggregate(lambda x: list(x))
df_deputados_profissoes['profissao'] = df_deputados_profissoes['profissao'].apply(lambda x: ','.join(map(str, x)))
df_deputados_profissoes['profissao'] = df_deputados_profissoes['profissao'].replace("nan", None, regex=True)


# Juntando profissoes com deputados
df_deputados = df_deputados.merge(df_deputados_profissoes, on='id_deputado', how='inner')

# Dados da API
df_deputados_api = pd.read_json(Path("data/api_files/deputados.json"), encoding = "ISO-8859-1")
df_deputados_api.columns = [change_col_names(i) for i in df_deputados_api.columns]
df_deputados_api['id_partido'] = df_deputados_api['uri_partido'].str.split('/').str[-1]
df_deputados_api = df_deputados_api[['id','id_partido', 'sigla_partido', 'email']]
df_deputados_api.rename(columns={"id": "id_deputado"}, inplace=True)

# Juntados deputados com dados da API
df_deputados = df_deputados.merge(df_deputados_api, on='id_deputado', how='inner')

# Salvando dados
dtypes_deputados = {
    'id_deputado':Integer(),
    'nome':String(),
    'id_legislatura_inicial':Integer(),
    'id_legislatura_final':Integer(),
    'nome_civil':String(),
    'sigla_sexo':String(),
    'data_nascimento':Date(),
    'data_falecimento':Date(),
    'uf_nascimento':String(),
    'municipio_nascimento':String(),
    'profissao':String(),
    'url_foto':String(),
    'email':String(),
    'id_partido':Integer(),
    'sigla_partido':String(),
}
df_deputados= df_deputados[dtypes_deputados.keys()]

nome_da_tabela = 'deputados'
df_deputados.to_sql(con =db_connection,
                    name=nome_da_tabela,
                    if_exists='replace',
                    index=False, 
                    method='multi',
                    dtype=dtypes_deputados
)


513

In [29]:
# Partidos
df_partidos = pd.read_json(Path("data/api_files/partidos.json"), encoding = "ISO-8859-1")
df_partidos.columns = [change_col_names(i) for i in df_partidos.columns]
df_partidos.drop(columns=['uri'], inplace=True)
df_partidos.rename(columns={"id": "id_partido"}, inplace=True)

df_partidos['id_partido'] = df_partidos['id_partido'].astype(int)

dtypes_partidos = {
    'id_partido':Integer(),
    'sigla':String(),
    'nome':String()
}
df_partidos= df_partidos[dtypes_partidos.keys()]

nome_da_tabela = 'partidos'
df_partidos.to_sql(con=db_connection,
                    name=nome_da_tabela,
                    if_exists='replace',
                    index=False, 
                    method='multi',
                    dtype=dtypes_partidos
)

23

In [30]:
# Legislaturas
df_legislaturas = pd.read_csv(Path("data/extrator_files/legislaturas.csv"), sep=';')
df_legislaturas.columns = [change_col_names(i) for i in df_legislaturas.columns]
df_legislaturas.drop(columns=['uri'], inplace=True)

df_legislaturas['id_legislatura'] = df_legislaturas['id_legislatura'].astype(int)
df_legislaturas['ano_eleicao'] = df_legislaturas['ano_eleicao'].astype(int)
df_legislaturas['data_inicio'] = pd.to_datetime(df_legislaturas['data_inicio'], format='%Y-%m-%d')
df_legislaturas['data_fim'] = pd.to_datetime(df_legislaturas['data_fim'], format='%Y-%m-%d')

dtypes_legislaturas = {
    "id_legislatura": Integer(),
    "data_inicio": Date(),
    "data_fim": Date(),
    "ano_eleicao": Integer()
}
df_legislaturas= df_legislaturas[dtypes_legislaturas.keys()]

nome_da_tabela = 'legislaturas'
df_legislaturas.to_sql(con=db_connection,
                       name=nome_da_tabela,
                       if_exists='replace',
                       index=False,
                       method='multi',
                       dtype=dtypes_legislaturas)

57

In [31]:
# Orgaos
df_orgaos = pd.read_csv(Path("data/extrator_files/orgaos.csv"), sep=';')

df_orgaos.columns = [change_col_names(i) for i in df_orgaos.columns]
df_orgaos.insert(0, 'id_orgao', df_orgaos['uri'].str.split('/').str[-1])
droped_cols = ['uri', 'nome_publicacao', 'cod_tipo_orgao', 'data_instalacao', 'data_fim_original', 'cod_situacao', 'url_website']
df_orgaos.drop(columns=droped_cols, inplace=True)

df_orgaos.rename(columns={"nome": "descricao", "apelido": "nome", "descricao_situacao": "situacao"}, inplace=True)
df_orgaos['data_inicio'] = pd.to_datetime(df_orgaos['data_inicio'], format='%Y-%m-%d')
df_orgaos['data_fim'] = pd.to_datetime(df_orgaos['data_fim'], format='%Y-%m-%d')

dtypes_orgaos = {
    'id_orgao':Integer(),
    'sigla':String(),
    'nome':String(),
    'descricao':String(),
    'tipo_orgao':String(),
    'data_inicio':Date(),
    'data_fim':Date(),
    'situacao':String(),
    'casa':String(),
    'sala':String()
}
df_orgaos = df_orgaos[dtypes_orgaos.keys()]

nome_da_tabela = 'orgaos'
df_orgaos.to_sql(con=db_connection,
                       name=nome_da_tabela,
                       if_exists='replace',
                       index=False,
                       method='multi',
                       dtype=dtypes_orgaos
)

3549

In [32]:
# Orgaos - Deputados
df_orgaos_deputados = pd.read_csv(Path("data/extrator_files/orgaosDeputados.csv"), sep=';')
df_orgaos_deputados.columns = [change_col_names(i) for i in df_orgaos_deputados.columns]
df_orgaos_deputados.insert(0, 'id_orgao', df_orgaos_deputados['uri_orgao'].str.split('/').str[-1])
df_orgaos_deputados.insert(0, 'id_deputado', df_orgaos_deputados['uri_deputado'].str.split('/').str[-1])
df_orgaos_deputados = df_orgaos_deputados[['id_orgao', 'id_deputado', 'cargo', 'data_inicio', 'data_fim']]

df_orgaos_deputados['data_inicio'] = pd.to_datetime(df_orgaos_deputados['data_inicio'], format='%Y-%m-%d')
df_orgaos_deputados['data_fim'] = pd.to_datetime(df_orgaos_deputados['data_fim'], format='%Y-%m-%d')

dtypes_orgaos_deputados = {
    'id_orgao':Integer(),
    'id_deputado':Integer(),
    'cargo':String(),
    'data_inicio':Date(),
    'data_fim':Date()
}
df_orgaos_deputados = df_orgaos_deputados[dtypes_orgaos_deputados.keys()]

nome_da_tabela = 'deputado_ocupacao_orgao'
df_orgaos_deputados.to_sql(con=db_connection,
                       name=nome_da_tabela,
                       if_exists='replace',
                       index=False,
                       method='multi',
                       dtype=dtypes_orgaos_deputados
)

14452

In [33]:
# Eventos
df_eventos = pd.read_csv(Path("data/extrator_files/eventos.csv"), sep=';')
df_eventos.columns = [change_col_names(i) for i in df_eventos.columns]
droped_cols = ['uri',
               'local_externo',
               'local_camara_nome',
               'local_camara_predio',
               'local_camara_sala',
               'local_camara_andar'
]
df_eventos.drop(columns=droped_cols, inplace=True)

df_eventos.rename(columns={"id": "id_evento", "data_hora_inicio": "datetime_inicio", "data_hora_fim": "datetime_fim"}, inplace=True)
df_eventos['datetime_inicio'] = pd.to_datetime(df_eventos['datetime_inicio'], format='%Y-%m-%dT%H:%M:%S')
df_eventos['datetime_fim'] = pd.to_datetime(df_eventos['datetime_fim'], format='%Y-%m-%dT%H:%M:%S')

dtypes_eventos = {
    'id_evento':Integer(),
    'situacao':String(),
    'descricao_tipo':String(),
    'descricao':String(),
    'datetime_inicio':DateTime(),
    'datetime_fim':DateTime(),
    'url_documento_pauta':String()
}
df_eventos = df_eventos[dtypes_eventos.keys()]

nome_da_tabela = 'eventos'
df_eventos.to_sql(con=db_connection,
                       name=nome_da_tabela,
                       if_exists='replace',
                       index=False,
                       method='multi',
                       dtype=dtypes_eventos
)

9320

In [34]:
# Eventos deputados
df_eventos_deputados = pd.read_csv(Path("data/extrator_files/eventosPresencaDeputados.csv"), sep=';')
df_eventos_deputados.columns = [change_col_names(i) for i in df_eventos_deputados.columns]
df_eventos_deputados = df_eventos_deputados[['id_evento', 'id_deputado', 'data_hora_inicio']]

df_eventos_deputados.rename(columns={"data_hora_inicio": "datetime_inicio"}, inplace=True)
df_eventos_deputados['datetime_inicio'] = pd.to_datetime(df_eventos_deputados['datetime_inicio'], format='%Y-%m-%dT%H:%M:%S')

dtypes_eventos_deputados = {
    'id_evento':Integer(),
    'id_deputado':Integer(),
    'datetime_inicio':DateTime()
}
df_eventos_deputados = df_eventos_deputados[dtypes_eventos_deputados.keys()]

nome_da_tabela = 'deputado_presenca_evento'
df_eventos_deputados.to_sql(con=db_connection,
                       name=nome_da_tabela,
                       if_exists='replace',
                       index=False,
                       method='multi',
                       dtype=dtypes_eventos_deputados
)

434854

In [35]:
# Votacoes
df_votacoes = pd.read_csv(Path("data/extrator_files/votacoes.csv"), sep=';')
df_votacoes.columns = [change_col_names(i) for i in df_votacoes.columns]
droped_cols = ['uri',
               'data_hora_registro',
               'uri_orgao',
               'sigla_orgao',
               'uri_evento',
               'ultima_abertura_votacao_data_hora_registro',
               'ultima_abertura_votacao_descricao',
               'ultima_apresentacao_proposicao_data_hora_registro',
               'ultima_apresentacao_proposicao_descricao',
               'ultima_apresentacao_proposicao_uri_proposicao',
]
df_votacoes.drop(columns=droped_cols, inplace=True)

df_votacoes.rename(columns={"id": "id_votacao", "data": "data_votacao", "aprovacao": "aprovado", "ultima_apresentacao_proposicao_id_proposicao":"ultima_proposicao_id"}, inplace=True)
df_votacoes['data_votacao'] = pd.to_datetime(df_votacoes['data_votacao'], format='%Y-%m-%d')
df_votacoes['ultima_proposicao_id'] = df_votacoes['ultima_proposicao_id'].replace('0', np.nan)

dtypes_votacoes = {
    'id_votacao':String(),
    'aprovado':Integer(),
    'votos_sim':Integer(),
    'votos_nao':Integer(),
    'votos_outros':Integer(),
    'descricao':String(),
    'data_votacao':Date(),
    'id_orgao':Integer(),
    'id_evento':Integer(),
    'ultima_proposicao_id':Integer()
}
df_votacoes = df_votacoes[dtypes_votacoes.keys()]

nome_da_tabela = 'votacoes'
df_votacoes.to_sql(con=db_connection,
                       name=nome_da_tabela,
                       if_exists='replace',
                       index=False,
                       method='multi',
                       dtype=dtypes_votacoes
)

25170

In [36]:
#Votações deputados
df_votacoes_deputados = pd.read_csv(Path("data/extrator_files/votacoesVotos.csv"), sep=';')
df_votacoes_deputados.columns = [change_col_names(i) for i in df_votacoes_deputados.columns]
droped_cols = ['uri_votacao',
               'deputado_uri',
               'deputado_nome',
               'deputado_sigla_partido',
               'deputado_uri_partido',
               'deputado_sigla_uf',
               'deputado_id_legislatura',
               'deputado_url_foto',
]
df_votacoes_deputados.drop(columns=droped_cols, inplace=True)

df_votacoes_deputados.rename(columns={"data_hora_voto": "datetime_voto", "deputado_id":"id_deputado"}, inplace=True)
df_votacoes_deputados['datetime_voto'] = pd.to_datetime(df_votacoes_deputados['datetime_voto'], format='%Y-%m-%dT%H:%M:%S')

dtypes_votacoes_deputados = {
    'id_votacao':String(),
    'id_deputado':Integer(),
    'voto':String(),
    'datetime_voto':DateTime()
}
df_votacoes_deputados = df_votacoes_deputados[dtypes_votacoes_deputados.keys()]

nome_da_tabela = 'deputado_voto_votacoes'
df_votacoes_deputados.to_sql(con=db_connection,
                       name=nome_da_tabela,
                       if_exists='replace',
                       index=False,
                       method='multi',
                       dtype=dtypes_votacoes_deputados
)

781147

In [37]:
# Frentes Parlamentares
df_frentes = pd.read_csv(Path("data/extrator_files/frentes.csv"), sep=';')
df_frentes.columns = [change_col_names(i) for i in df_frentes.columns]
df_frentes = df_frentes[df_frentes['id_legislatura'] == 56]
df_frentes = df_frentes[['id', 'titulo', 'data_criacao', 'coordenador_id']]


df_frentes.rename(columns={"id": "frente_id", "coordenador_id":"deputado_coordenador_id"}, inplace=True)
df_frentes['data_criacao'] = pd.to_datetime(df_frentes['data_criacao'], format='%Y-%m-%d')

dtypes_frentes = {
    'frente_id':Integer(),
    'titulo':String(),
    'data_criacao':Date(),
    'deputado_coordenador_id':Integer()
}
df_frentes = df_frentes[dtypes_frentes.keys()]

nome_da_tabela = 'frentes_parlamentares'
df_frentes.to_sql(con=db_connection,
                       name=nome_da_tabela,
                       if_exists='replace',
                       index=False,
                       method='multi',
                       dtype=dtypes_frentes
)

352

In [38]:
# Frentes Parlamentares
df_frentes_deputados = pd.read_csv(Path("data/extrator_files/frentesDeputados.csv"), sep=';')
df_frentes_deputados.columns = [change_col_names(i) for i in df_frentes_deputados.columns]
df_frentes_deputados = df_frentes_deputados[df_frentes_deputados['deputado__id_legislatura'] == 56]
df_frentes_deputados = df_frentes_deputados[['id', 'deputado__id', 'deputado__titulo']]
df_frentes_deputados.rename(columns={"id": "frente_id", "deputado__id":"deputado_id", "deputado__titulo":"deputado_cargo"}, inplace=True)

dtypes_frentes_deputados = {
    'frente_id':Integer(),
    'deputado_id':Integer(),
    'deputado_cargo':String()
}
df_frentes_deputados = df_frentes_deputados[dtypes_frentes_deputados.keys()]

nome_da_tabela = 'deputado_ocupacao_frente'
df_frentes_deputados.to_sql(con=db_connection,
                       name=nome_da_tabela,
                       if_exists='replace',
                       index=False,
                       method='multi',
                       dtype=dtypes_frentes_deputados
)

75593

In [39]:
# Frentes Parlamentares
df_frentes_deputados = pd.read_csv(Path("data/extrator_files/frentesDeputados.csv"), sep=';')
df_frentes_deputados.columns = [change_col_names(i) for i in df_frentes_deputados.columns]
df_frentes_deputados = df_frentes_deputados[df_frentes_deputados['deputado__id_legislatura'] == 56]
df_frentes_deputados = df_frentes_deputados[['id', 'deputado__id', 'deputado__titulo']]
df_frentes_deputados.rename(columns={"id": "frente_id", "deputado__id":"deputado_id", "deputado__titulo":"deputado_cargo"}, inplace=True)

dtypes_frentes_deputados = {
    'frente_id':Integer(),
    'deputado_id':Integer(),
    'deputado_cargo':String()
}
df_frentes_deputados = df_frentes_deputados[dtypes_frentes_deputados.keys()]

nome_da_tabela = 'deputado_ocupacao_frente'
df_frentes_deputados.to_sql(con=db_connection,
                       name=nome_da_tabela,
                       if_exists='replace',
                       index=False,
                       method='multi',
                       dtype=dtypes_frentes_deputados
)

75593

In [4]:
df_proposicoes = pd.read_csv(Path("data/extrator_files/proposicoes.csv"), sep=';')
df_proposicoes.columns = [change_col_names(i) for i in df_proposicoes.columns]
proposicoes_cols = [
    'id',
    'sigla_tipo',
    'descricao_tipo',
    'ano',
    'ementa',
    'ementa_detalhada',
    'keywords',
    'data_apresentacao',
    'uri_prop_principal',
    'url_inteiro_teor',
    'ultimo_status_data_hora',
    'ultimo_status_sequencia',
    'ultimo_status_id_orgao',
    'ultimo_status_regime',
    'ultimo_status_descricao_tramitacao',
    'ultimo_status_descricao_situacao',
    'ultimo_status_despacho',
    'ultimo_status_url'
]
df_proposicoes = df_proposicoes[proposicoes_cols]
df_proposicoes['id_proposicao_principal'] = df_proposicoes['uri_prop_principal'].str.split('/').str[-1]
df_proposicoes['datetime_apresentacao'] = pd.to_datetime(df_proposicoes['data_apresentacao'], format='%Y-%m-%dT%H:%M:%S')
df_proposicoes['ultimo_status_datetime'] = pd.to_datetime(df_proposicoes['ultimo_status_data_hora'], format='%Y-%m-%dT%H:%M:%S')
df_proposicoes.rename(columns={"id": "id_proposicao", "ano": "ano_apresentacao"}, inplace=True)

# Pegando os temas da proposicoes
df_proposicoes_temas = pd.read_csv(Path("data/extrator_files/proposicoesTemas.csv"), sep=';')
df_proposicoes_temas.columns = [change_col_names(i) for i in df_proposicoes_temas.columns]
df_proposicoes_temas.insert(0, 'id_proposicao', df_proposicoes_temas['uri_proposicao'].str.split('/').str[-1])
df_proposicoes_temas['id_proposicao'] = df_proposicoes_temas['id_proposicao'].astype('Int64')
df_proposicoes_temas = df_proposicoes_temas[['id_proposicao', 'cod_tema', 'tema']]

# Juntados deputados com dados dos temas
df_proposicoes = df_proposicoes.merge(df_proposicoes_temas, on='id_proposicao', how='left')

# Pegando os dados dos autores das proposicoes
df_proposicoes_autores = pd.read_csv(Path("data/extrator_files/proposicoesAutores.csv"), sep=';')
df_proposicoes_autores.columns = [change_col_names(i) for i in df_proposicoes_autores.columns]
df_proposicoes_autores['id_deputado_autor'] = df_proposicoes_autores['id_deputado_autor']
df_proposicoes_autores = df_proposicoes_autores[['id_proposicao', 'id_deputado_autor', 'tipo_autor', 'nome_autor']]
df_proposicoes_autores

# Juntados deputados con autores
df_proposicoes = df_proposicoes.merge(df_proposicoes_autores, on='id_proposicao', how='left')

# Salvando dados
dtypes_proposicoes = {
    'id_proposicao':Integer(),
    'sigla_tipo':String(),
    'descricao_tipo':String(),
    'ementa':String(),
    'ementa_detalhada':String(),
    'cod_tema':Integer(),
    'tema':String(),
    'keywords':String(),
    'id_deputado_autor':Integer(),
    'tipo_autor':String(),
    'nome_autor':String(),
    'datetime_apresentacao':DateTime(),
    'ano_apresentacao':Integer(),
    'id_proposicao_principal':Integer(),
    'url_inteiro_teor':String(),
    'ultimo_status_datetime':DateTime(),
    'ultimo_status_sequencia':Integer(),
    'ultimo_status_id_orgao':Integer(),
    'ultimo_status_regime':String(),
    'ultimo_status_descricao_tramitacao':String(),
    'ultimo_status_descricao_situacao':String(),
    'ultimo_status_despacho':String(),
    'ultimo_status_url':String(),
}
df_proposicoes= df_proposicoes[dtypes_proposicoes.keys()]



C:\Users\Edson Junior\AppData\Local\Temp\ipykernel_15244\3360215363.py:1: DtypeWarning: Columns (8,9,14,19,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_proposicoes = pd.read_csv(Path("data/extrator_files/proposicoes.csv"), sep=';')
C:\Users\Edson Junior\AppData\Local\Temp\ipykernel_15244\3360215363.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_proposicoes['id_proposicao_principal'] = df_proposicoes['uri_prop_principal'].str.split('/').str[-1]
C:\Users\Edson Junior\AppData\Local\Temp\ipykernel_15244\3360215363.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [7]:
df_proposicoes.shape

(349273, 23)

In [8]:
nome_da_tabela = 'proposicoes'
df_proposicoes.to_sql(con =db_connection,
                    name=nome_da_tabela,
                    if_exists='replace',
                    index=False, 
                    method='multi',
                    dtype=dtypes_proposicoes,
                    chunksize=10000
)

349273